In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import re # for regex
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
data['sentiment'].value_counts()

In [ ]:
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
data['review'][0]

<h2>Steps to clean the Reviews</h2>
    1. Remove HTML tags<br>
2. Remove special characters<br>
3. Convert everything to lowercase<br>
4. Remove stopwords<br>
5. Stemming<br>

<h3> Remove HTML tags

In [ ]:
def clean_text(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

In [ ]:
data['review'] = data['review'].apply(clean_text)
data['review'][0]

<h3> Remove Special Characters

In [ ]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

In [ ]:
data['review'] = data['review'].apply(is_special)
data['review'][0]

<h3> Convert everything to lowercase

In [ ]:
def to_lower(text):
    return text.lower()

In [ ]:
data['review'] = data['review'].apply(to_lower)
data['review'][0]

<h3> Remove Stopwords

In [ ]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

In [ ]:
data['review'] = data['review'].apply(rem_stopwords)
data['review'][0]

<h3> Words Stemming

In [ ]:
def stem_text(text):
    ss = SnowballStemmer('english')
    return ' '.join([ss.stem(w) for w in text])

In [ ]:
data['review'] = data['review'].apply(stem_text)
data['review'][0]

<h2> Create Words Clouds for positive and negative reviews

In [ ]:
positive_df = data[data['sentiment'] == 1]
negative_df = data[data['sentiment'] == 0]

In [ ]:
stop_words = set(STOPWORDS)
def words_for_cloud(df):
    words = ''
    for val in df['review'].loc[10:20]:
        val = str(val)
        tokens = val.split()
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
        words += " ".join(tokens)+" "
    return words

In [ ]:
# Used only first 10 reviews to plot the word cloud
pos_words = words_for_cloud(positive_df)
neg_words = words_for_cloud(negative_df)

pos_wordcloud = WordCloud(width = 800, height = 800,background_color ='white', stopwords = stop_words, min_font_size = 10).generate(pos_words)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(pos_wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

neg_wordcloud = WordCloud(width = 800, height = 800,background_color ='white', stopwords = stop_words, min_font_size = 10).generate(neg_words)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(neg_wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

<h2> Creating the Model

<h3> Creating the Bag of Words

In [ ]:
X = np.array(data.iloc[:,0].values)
y = np.array(data['sentiment'].values)
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(data.review).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

<h3>Train Test Split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=9)

In [ ]:
print("Train shapes : X = {}, y = {}".format(train_X.shape, train_y.shape))
print("Test shapes : X = {}, y = {}".format(test_X.shape, test_y.shape))

<h3> Defining and Training the Models </h3>
Used Bernoulli's Naive Bayes classifier. This classifiers is suitable for discrete data and works best for Binary features.


In [ ]:
bernoulli = BernoulliNB(alpha=1.0,fit_prior=True)
bernoulli.fit(train_X, train_y)

<h3> Prediction and accuracy scores

In [ ]:
y_predict_bernoulli = bernoulli.predict(test_X)

In [ ]:
print("Bernoulli = ",accuracy_score(test_y,y_predict_bernoulli)*100)

<h3> Test run on a dummy review

In [ ]:
rev =  """Terrible. Complete trash. Brainless tripe. Insulting to anyone who isn't an 8 year old fan boy. Im actually pretty disgusted that this movie is making the money it is - what does it say about the people who brainlessly hand over the hard earned cash to be 'entertained' in this fashion and then come here to leave a positive 8.8 review?? Oh yes, they are morons. Its the only sensible conclusion to draw. How anyone can rate this movie amongst the pantheon of great titles is beyond me.

So trying to find something constructive to say about this title is hard...I enjoyed Iron Man? Tony Stark is an inspirational character in his own movies but here he is a pale shadow of that...About the only 'hook' this movie had into me was wondering when and if Iron Man would knock Captain America out...Oh how I wished he had :( What were these other characters anyways? Useless, bickering idiots who really couldn't organise happy times in a brewery. The film was a chaotic mish mash of action elements and failed 'set pieces'...

I found the villain to be quite amusing.

And now I give up. This movie is not robbing any more of my time but I felt I ought to contribute to restoring the obvious fake rating and reviews this movie has been getting on IMDb."""

In [ ]:
f1 = clean_text(rev)
f2 = is_special(f1)
f3 = to_lower(f2)
f4 = rem_stopwords(f3)
f5 = stem_text(f4)

bow, words = [], word_tokenize(f5)
for word in words:
    bow.append(words.count(word))
    
word_dict = cv.vocabulary_

inp = []
for i in word_dict:
    inp.append(f5.count(i[0]))
y_pred = bernoulli.predict(np.array(inp).reshape(1,1000))

if y_pred[0] == 1:
    print("Review is Positive")
else:
    print("Review is Negative")